In [ ]:
import os
import nibabel as nib
import numpy as np
import torch
from torch.utils.data import Dataset
import shutil
import random
import shutil

# Préparation des données recadrage
data_dir = 'data/BraTS2021_Training_Data/'  # Dossier principal contenant les données
save_dir = 'BraTS2021_Cropped/'  # Dossier où sauvegarder les images recadrées
os.makedirs(save_dir, exist_ok=True)

# Fonction de recadrage aléatoire appliqué de manière uniforme
def random_crop(image, start_coords, new_shape=(128, 128, 128)):
    """ Recadre une image en utilisant les mêmes coordonnées pour toutes les modalités """
    start_z, start_y, start_x = start_coords
    target_z, target_y, target_x = new_shape

    end_z, end_y, end_x = start_z + target_z, start_y + target_y, start_x + target_x

    return image[start_z:end_z, start_y:end_y, start_x:end_x]

#  Parcourir les dossiers des patients
for patient in os.listdir(data_dir):
    patient_path = os.path.join(data_dir, patient)

    if os.path.isdir(patient_path):
        print(f" Traitement de {patient}...")

        #  Charger les images avec l'extension .nii
        t1_path = os.path.join(patient_path, f'{patient}_t1.nii.gz')
        t1ce_path = os.path.join(patient_path, f'{patient}_t1ce.nii.gz')
        t2_path = os.path.join(patient_path, f'{patient}_t2.nii.gz')
        flair_path = os.path.join(patient_path, f'{patient}_flair.nii.gz')
        label_path = os.path.join(patient_path, f'{patient}_seg.nii.gz')

        has_label = os.path.exists(label_path)

        t1_img = nib.load(t1_path)
        t1ce_img = nib.load(t1ce_path)
        t2_img = nib.load(t2_path)
        flair_img = nib.load(flair_path)

        if has_label:
            label_img = nib.load(label_path)

        #  Déterminer un recadrage unique pour toutes les modalités
        z, y, x = t1_img.shape
        target_z, target_y, target_x = (128, 128, 128)
        start_z = np.random.randint(0, z - target_z + 1)
        start_y = np.random.randint(0, y - target_y + 1)
        start_x = np.random.randint(0, x - target_x + 1)

        start_coords = (start_z, start_y, start_x)

        #  Appliquer le même recadrage à toutes les images du patient
        t1_cropped = random_crop(t1_img.get_fdata(), start_coords)
        t1ce_cropped = random_crop(t1ce_img.get_fdata(), start_coords)
        t2_cropped = random_crop(t2_img.get_fdata(), start_coords)
        flair_cropped = random_crop(flair_img.get_fdata(), start_coords)

        if has_label:
            label_cropped = random_crop(label_img.get_fdata(), start_coords)

        #  Créer dossier patient dans le dossier de sauvegarde
        save_patient_dir = os.path.join(save_dir, patient)
        os.makedirs(save_patient_dir, exist_ok=True)

        #  Sauvegarder les images recadrées
        nib.save(nib.Nifti1Image(t1_cropped, t1_img.affine), os.path.join(save_patient_dir, f'{patient}_t1.nii.gz'))
        nib.save(nib.Nifti1Image(t1ce_cropped, t1ce_img.affine), os.path.join(save_patient_dir, f'{patient}_t1ce.nii.gz'))
        nib.save(nib.Nifti1Image(t2_cropped, t2_img.affine), os.path.join(save_patient_dir, f'{patient}_t2.nii.gz'))
        nib.save(nib.Nifti1Image(flair_cropped, flair_img.affine), os.path.join(save_patient_dir, f'{patient}_flair.nii.gz'))

        if has_label:
            nib.save(nib.Nifti1Image(label_cropped, label_img.affine), os.path.join(save_patient_dir, f'{patient}_seg.nii.gz'))

        print(f"{patient} traité et sauvegardé.")

print(" Fin du recadrage !")



In [ ]:
# Normalisation des données
data_dir = "BraTS2021_Cropped"

#  Dossier de sauvegarde des images normalisées
output_dir = "BraTS2021_Normalized"
os.makedirs(output_dir, exist_ok=True)

#  Fonction de normalisation Z-score (uniquement sur les voxels non nuls)
def normalize_image(image):
    non_zero_mask = image != 0  # Détecter les voxels non nuls
    non_zero_values = image[non_zero_mask]  # Extraire les valeurs non nulles

    if len(non_zero_values) == 0:
        return image

    mean = np.mean(non_zero_values)
    std = np.std(non_zero_values)


    # Normalisation
    normalized_image = image.copy()
    normalized_image[non_zero_mask] = (non_zero_values - mean) / std

    return normalized_image


# Parcourir les dossiers des patients
for patient in os.listdir(data_dir):
    patient_dir = os.path.join(data_dir, patient)
    if os.path.isdir(patient_dir):
        save_patient_dir = os.path.join(output_dir, patient)
        os.makedirs(save_patient_dir, exist_ok=True)

        #  Parcourir les fichiers du patient
        for file in os.listdir(patient_dir):
            file_path = os.path.join(patient_dir, file)

            # Ignorer les fichiers de segmentation
            if "seg" in file:
                shutil.copy(file_path, os.path.join(save_patient_dir, file))
                print(f" {file} copié sans modification.")
                continue

            # Charger et normaliser l'image
            img = nib.load(file_path)
            data = img.get_fdata()
            normalized_data = normalize_image(data)

            # Sauvegarde de l'image normalisée
            nib.save(nib.Nifti1Image(normalized_data, img.affine), os.path.join(save_patient_dir, file))
            print(f" {file} normalisé et sauvegardé.")

print("\n Normalisation terminée !")

In [ ]:
#Division des données


normalized_data_dir = "BraTS2021_Normalized"
train_dir = "BraTS2021_Train"  # Dossier pour les données d'entraînement
val_dir = "BraTS2021_Val"  # Dossier pour les données de validation
test_dir = "BraTS2021_Test"  # Dossier pour les données de test


os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Fonction pour diviser les données par dossier
def split_data_by_folder(data_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):

    all_folders = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]

    # Mélanger aléatoirement les sous-dossiers
    random.shuffle(all_folders)


    total_folders = len(all_folders)
    train_count = int(total_folders * train_ratio)
    val_count = int(total_folders * val_ratio)
    test_count = total_folders - train_count - val_count

    # Séparer les sous-dossiers en ensembles
    train_folders = all_folders[:train_count]
    val_folders = all_folders[train_count:train_count + val_count]
    test_folders = all_folders[train_count + val_count:]

    # Déplacer les sous-dossiers dans leurs dossiers respectifs
    def move_folders(folder_list, destination_dir):
        for folder in folder_list:
            src_folder = os.path.join(data_dir, folder)
            dest_folder = os.path.join(destination_dir, folder)
            shutil.copytree(src_folder, dest_folder)

    # Déplacer les sous-dossiers
    move_folders(train_folders, train_dir)
    move_folders(val_folders, val_dir)
    move_folders(test_folders, test_dir)

    print(f" Données divisées en : {train_count} dossiers pour l'entraînement, {val_count} pour la validation et {test_count} pour le test.")

# Diviser les données normalisées par dossier
split_data_by_folder(normalized_data_dir)

In [ ]:
import os
import shutil
import nibabel as nib
import numpy as np
import random

#  Dossiers
input_dir = "BraTS2021_Train"
output_dir = "BraTS2021_Augmented"
os.makedirs(output_dir, exist_ok=True)

#  Fonction pour appliquer un flip selon les axes
def flip_image(image, axes=(False, False, False)):
    if axes[0]:  # Flip axial
        image = np.flip(image, axis=0)
    if axes[1]:  # Flip coronal
        image = np.flip(image, axis=1)
    if axes[2]:  # Flip sagittal
        image = np.flip(image, axis=2)
    return image

#  Fonction pour appliquer un décalage d'intensité
def intensity_shift(image, factor=0.1):
    shift = np.random.uniform(-factor, factor) * np.mean(image)
    return image + shift

#  Parcours des dossiers des patients
for patient in os.listdir(input_dir):
    patient_dir = os.path.join(input_dir, patient)
    if os.path.isdir(patient_dir):
        #  Créer le dossier original dans BraTS2021_Augmented
        original_dir = os.path.join(output_dir, patient)
        os.makedirs(original_dir, exist_ok=True)
        shutil.copytree(patient_dir, original_dir, dirs_exist_ok=True)

        #  Générer 2 augmentations différentes
        for aug_idx in range(1, 3):
            augmented_dir = os.path.join(output_dir, f"{patient}_aug{aug_idx}")
            os.makedirs(augmented_dir, exist_ok=True)

            #  Générer une combinaison de flips aléatoire
            flip_axes = (
                random.random() < 0.5,  # 50% de chance de flip axial
                random.random() < 0.5,  # 50% de chance de flip coronal
                random.random() < 0.5   # 50% de chance de flip sagittal
            )

            #  Appliquer les augmentations
            for file in os.listdir(patient_dir):
                filepath = os.path.join(patient_dir, file)
                if file.endswith(".nii.gz"):
                    img = nib.load(filepath)
                    data = img.get_fdata()
                    affine = img.affine

                    #  Segmentation: uniquement flip
                    if "seg" in file:
                        augmented_data = flip_image(data, flip_axes)
                    else:
                        #  Autres modalités: flip + décalage d'intensité
                        augmented_data = intensity_shift(flip_image(data, flip_axes))


                    parts = file.split('_')
                    new_filename = f"{parts[0]}_{parts[1]}_aug{aug_idx}_{parts[2]}"
                    augmented_img = nib.Nifti1Image(augmented_data, affine)
                    nib.save(augmented_img, os.path.join(augmented_dir, new_filename))

print("Augmentation terminée avec 2 versions par patient !")